# European Centre for Medium Range Weather Forecasts

Presentado por: Yosef Guevara Salamanca

## 1. Descarga del script

[Download script](https://gist.github.com/mrocklin/26d8323f9a8a6a75fce0)

Download historical global weather data from the ECMWF.

In [1]:
import pandas as pd
dates = pd.date_range('2014-01-01', '2014-12-31', freq='D')
dates = [str(d).split()[0] for d in dates]

from ecmwfapi import ECMWFDataServer

server = ECMWFDataServer()

for date in dates:
    server.retrieve({
      'stream'    : "oper",
      'levtype'   : "sfc",
      'param'     : "165.128/166.128/167.128",
      'dataset'   : "interim",
      'step'      : "00",
      'grid'      : "0.25/0.25",
      'time'      : "00/06/12/18",
      'date'      : date,
      'type'      : "an",
      'class'     : "ei",
      'target'    : date + ".nc3",
      'format'    : "netcdf" })

2021-07-27 15:43:47 ECMWF API python library 1.6.1
2021-07-27 15:43:47 ECMWF API at https://api.ecmwf.int/v1
2021-07-27 15:43:48 Welcome anonymous
2021-07-27 15:43:49 
2021-07-27 15:43:49 You are accessing ECMWF data services as an anonymous user. For
2021-07-27 15:43:49 improved quality of service you should consider registering an account
2021-07-27 15:43:49 with ECMWF.
2021-07-27 15:43:49 


APIException: "ecmwf.API error 1: User 'anonymous' has not access to datasets/interim. Please accept the terms and conditions at http://apps.ecmwf.int/datasets/licences/general"

**Try the following:**

*  What is the variance in temperature over time?
*  What areas experienced the largest temperature swings in the last month relative to their previous history?
*  Plot the temperature of the earth as a function of latitude and then as longitude